# Create Node and Edge Files for Gephi Import
## This will create a ***nodes.csv*** and ***edges.csv*** file

### Requires:
* master_matches.db
* master_icw.csv
* tree_attributes.csv

In [1]:
import pandas as pd
import sqlite3

## Set Variables

In [2]:
# file_path is the path to the directory that your files exist in if not in same directory
file_path = "../files/"

## Connect to *master_matches.db* and Create a DataFrame

In [3]:
# Connect to the SQLite database
conn_read = sqlite3.connect(f'{file_path}master_matches.db')

# Read data from a table into a DataFrame
master_df = pd.read_sql_query('SELECT * FROM matches', conn_read)

# Close the connection
conn_read.close()

master_df.head(1)

,ancestry,match_name,match_admin,sharedCM,segments,side,mrca,gens,notes
0,anc_id 1,match_name 1,match_admin 1,3475.0,24,M,mrca 1,1.0,None


## Connect to ***master_icw.csv*** and Create DataFrame

In [10]:
icw_df = pd.read_csv(f'{file_path}master_icw.csv')
             
icw_df.head(1)

,source_id,target_id
0,anc_id 248,anc_id 769


## Connect to ***tree_attributes.csv*** and Create DataFrame

In [4]:
attributes_df = pd.read_csv(f'{file_path}tree_attributes.csv', usecols=['gens', 'color','x', 'y'])
             
attributes_df.head(1)

,gens,color,x,y
0,1.0,fffdd0,0.0,0.0


## Clean String Data and Add a Side Column

In [11]:
icw_df['source_id'] = icw_df['source_id'].str.lower().str.strip()
icw_df['target_id'] = icw_df['target_id'].str.lower().str.strip()

master_df['ancestry'] = master_df['ancestry'].str.lower().str.strip()

## Filter *matches_df* to Create a Subset of Nodes to Include in ***nodes.csv***

In [8]:
# Create Filters to Subset the Nodes List Based on Determined Criteria
# Filter on cm_size, known_mrca, side, etc. 
# Then create a node list after filter
node_list = []

node_list = master_df["ancestry"].unique()

master_df.reset_index(drop=True)
master_df["id"] = master_df.index

master_df.head(1)

,ancestry,match_name,match_admin,sharedCM,segments,side,mrca,gens,notes,id
0,anc_id 1,match_name 1,match_admin 1,3475.0,24,M,mrca 1,1.0,None,0


In [9]:
node_dict = {}

   
# Iterate through dataframe
for index, row in master_df.iterrows():
    if row['ancestry'] in node_list:
        node_dict[row['ancestry']] = row['id']

## Filter icw_df to Include Filtered Nodes Only

In [12]:
# Filter df_icw to only those that both match_ids are in list

# Map icw_df['matchid'] and icw_df['icwid'] to icw_df['source'] and icw_df['target']:
icw_df['source'] = icw_df['source_id'].map(node_dict)
icw_df['target'] = icw_df['target_id'].map(node_dict)

icw_df.dropna(subset = ['source', 'target'], inplace=True)
icw_df[['source','target']] = icw_df[['source','target']].astype('int16')

icw_df.head(1)

,source_id,target_id,source,target
0,anc_id 248,anc_id 769,247,768


## Join *master_df*  and *attributes_df* on *gens*

In [14]:
master_df = master_df.merge(attributes_df, how='left', on='gens')

master_df.head(1)

,ancestry,match_name,match_admin,sharedCM,segments,side,mrca,gens,notes,id,color,x,y
0,anc_id 1,match_name 1,match_admin 1,3475.0,24,M,mrca 1,1.0,None,0,fffdd0,0.0,0.0


## Create Label Columns and Select Columns for Export

In [15]:
#Select columnns for node file and rename
master_df['label'] = master_df['match_name'] + " " + master_df['match_admin']
master_df['gens_int'] = master_df['gens'].astype('int8')
nodes_df = (master_df[['id','label','ancestry','color','x','y','sharedCM',
                       'segments','side','mrca','gens','gens_int']]
           )

nodes_df.head(1)

,id,label,ancestry,color,x,y,sharedCM,segments,side,mrca,gens,gens_int
0,0,match_name 1 match_admin 1,anc_id 1,fffdd0,0.0,0.0,3475.0,24,M,mrca 1,1.0,1


In [16]:
#Select columns for edge file
edges_df = icw_df[['source','target']]

edges_df.head(1)

,source,target
0,247,768


## Export DataFrames to CSV Files

In [17]:
nodes_df.to_csv(f'{file_path}nodes.csv', index=False)

edges_df.to_csv(f'{file_path}edges.csv', index=False)

### Features to Add

1. Create a series of filters to create new gephi import files
    * Filter by cm size
    * Filter by finding matches of certain matches to graph a specific side
    * Filter by known_mrca
    * Filter by side